## Librerias

https://boxuancui.github.io/DataExplorer/index.html

In [1]:
# Librerias
require("data.table")
require("rpart")
require("rpart.plot")
library("caret")

Loading required package: data.table

Loading required package: rpart

Loading required package: rpart.plot

Loading required package: ggplot2

Loading required package: lattice



In [2]:
library(pastecs)
library(summarytools)
library(DataExplorer)
library(dplyr)
library(kableExtra)


Attaching package: 'pastecs'


The following objects are masked from 'package:data.table':

    first, last



Attaching package: 'dplyr'


The following objects are masked from 'package:pastecs':

    first, last


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'kableExtra'


The following object is masked from 'package:dplyr':

    group_rows




In [12]:
library(ggplot2)
library(gridExtra)
require(dplyr)

In [3]:
# cargo el dataset
dataset <- fread("c:\\Users\\Usuario\\Documents\\Universidad\\austral\\2024\\lab1\\tps\\datasets\\dataset_pequeno.csv")

In [4]:
dtrain <- dataset[foto_mes == 202107] # defino donde voy a entrenar
dtest <- dataset[foto_mes == 202109] # defino donde voy a aplicar el modelo

# Análisis Exploratorio de Datos

In [ ]:
head(dtrain)

In [ ]:
tail(dtrain)

## Introducción: filas, columnas, valores faltantes.

In [17]:
introduce(dtrain)

rows,columns,discrete_columns,continuous_columns,all_missing_columns,total_missing_values,complete_rows,total_observations,memory_usage
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
164596,155,1,154,0,1648475,264,25512380,150805440


In [16]:
pdf("introduccion.pdf", width = 15, height = 15) 
plot_intro(dtrain)
dev.off()

png 
  2

## Descripción

<i>Es lo mismo que hacer : stat.desc(dtrain)</i>

In [34]:
resumen <- summarytools::descr(dtrain, transpose = TRUE)

In [51]:
head(resumen)

In [49]:
# Quito la notacion cientifica
numeric_cols <- sapply(resumen, is.numeric)
resumen[numeric_cols] <- lapply(resumen[, numeric_cols], function(x) format(x, scientific = FALSE))

kbl(resumen, escape = FALSE) %>%
  kable_paper() %>%
  save_kable(file = "descripcion.html", self_contained = F)

## Tipos de datos

In [10]:
str(dtrain)

Classes 'data.table' and 'data.frame':	164596 obs. of  155 variables:
 $ numero_de_cliente                   : int  29183733 29184468 29185245 29186441 29186475 29187730 29187764 29187961 29189899 29189993 ...
 $ foto_mes                            : int  202107 202107 202107 202107 202107 202107 202107 202107 202107 202107 ...
 $ active_quarter                      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ cliente_vip                         : int  0 0 0 0 0 0 0 0 0 0 ...
 $ internet                            : int  0 0 0 0 0 0 0 0 0 0 ...
 $ cliente_edad                        : int  46 42 55 62 66 65 56 62 60 59 ...
 $ cliente_antiguedad                  : int  99 284 23 296 326 326 91 326 379 74 ...
 $ mrentabilidad                       : num  2016 1634 3760 2693 4568 ...
 $ mrentabilidad_annual                : num  32147 13782 24297 75391 56306 ...
 $ mcomisiones                         : num  1099 603 2829 2494 3691 ...
 $ mactivos_margen                     : num  -113 -241 746 -2909

In [9]:
y <- lapply(dtrain, class)

unlist(y, use.names = TRUE)

numero_de_cliente                             foto_mes 
                           "integer"                            "integer" 
                      active_quarter                          cliente_vip 
                           "integer"                            "integer" 
                            internet                         cliente_edad 
                           "integer"                            "integer" 
                  cliente_antiguedad                        mrentabilidad 
                           "integer"                            "numeric" 
                mrentabilidad_annual                          mcomisiones 
                           "numeric"                            "numeric" 
                     mactivos_margen                      mpasivos_margen 
                           "numeric"                            "numeric" 
                          cproductos                             tcuentas 
                           "integer"                            "integer" 
                   ccuenta_corriente          mcuenta_corriente_adicional 
                           "integer"                            "numeric" 
                   mcuenta_corriente                         ccaja_ahorro 
                           "numeric"                            "integer" 
                        mcaja_ahorro               mcaja_ahorro_adicional 
                           "numeric"                            "numeric" 
                mcaja_ahorro_dolares             cdescubierto_preacordado 
                           "numeric"                            "integer" 
                      mcuentas_saldo                      ctarjeta_debito 
                           "numeric"                            "integer" 
       ctarjeta_debito_transacciones                        mautoservicio 
                           "integer"                            "numeric" 
                       ctarjeta_visa          ctarjeta_visa_transacciones 
                           "integer"                            "integer" 
               mtarjeta_visa_consumo                      ctarjeta_master 
                           "numeric"                            "integer" 
       ctarjeta_master_transacciones              mtarjeta_master_consumo 
                           "integer"                            "numeric" 
               cprestamos_personales                mprestamos_personales 
                           "integer"                            "numeric" 
               cprestamos_prendarios                mprestamos_prendarios 
                           "integer"                            "numeric" 
             cprestamos_hipotecarios              mprestamos_hipotecarios 
                           "integer"                            "numeric" 
                         cplazo_fijo                  mplazo_fijo_dolares 
                           "integer"                            "numeric" 
                   mplazo_fijo_pesos                          cinversion1 
                           "numeric"                            "integer" 
                   minversion1_pesos                  minversion1_dolares 
                           "numeric"                            "numeric" 
                         cinversion2                          minversion2 
                           "integer"                            "numeric" 
                        cseguro_vida                         cseguro_auto 
                           "integer"                            "integer" 
                    cseguro_vivienda        cseguro_accidentes_personales 
                           "integer"                            "integer" 
                     ccaja_seguridad                         cpayroll_trx 
                           "integer"                            "integer" 
                            mpayroll                            mpayroll2 
                           "numeric"        

## Valores faltantes

In [23]:
pdf("missing_values.pdf", width = 15, height = 15) 
plot_missing(
  dtrain,
  group = list(Good = 0.05, OK = 0.4, Bad = 0.8, Remove = 1),
  group_color = list(Good = "#1B9E77", OK = "#E6AB02", Bad = "#D95F02", Remove =
    "#E41A1C"),
  missing_only = TRUE,
  geom_label_args = list(),
  title = NULL,
  ggtheme = theme_gray(),
  theme_config = list(legend.position = c("bottom"))
)
dev.off()

png 
  2

In [ ]:
dfalt <- profile_missing(dtrain)
dfalt <- dfalt[dfalt$num_missing != 0]
dfalt %>% arrange(desc(num_missing))

In [171]:
# Sumar los valores faltantes por columna
missing_counts <- colSums(is.na(dtrain))

# Ordenar los recuentos de valores faltantes de manera descendente
missing_counts_sorted <- sort(missing_counts, decreasing = TRUE)

missing_counts_sorted

Master_Finiciomora                     Visa_Finiciomora 
                              163858                               163595 
               Master_mconsumospesos              Master_mconsumosdolares 
                               94804                                94804 
               Master_madelantopesos              Master_madelantodolares 
                               94804                                94804 
                  Master_mpagospesos                 Master_mpagosdolares 
                               94804                                94804 
                Master_mconsumototal                     Master_cconsumos 
                               94804                                94804 
           Master_cadelantosefectivo                  Visa_mconsumospesos 
                               94804                                20333 
               Visa_mconsumosdolares                  Visa_madelantopesos 
                               20333                                20333 
               Visa_madelantodolares                     Visa_mpagospesos 
                               20333                                20333 
                  Visa_mpagosdolares                   Visa_mconsumototal 
                               20333                                20333 
                      Visa_cconsumos              Visa_cadelantosefectivo 
                               20333                                20333 
               Master_fultimo_cierre                   Master_delinquency 
                               15781                                15743 
                       Master_status          Master_mfinanciacion_limite 
                               15743                                15743 
                 Master_Fvencimiento                   Master_msaldototal 
                               15743                                15743 
                  Master_msaldopesos                 Master_msaldodolares 
                               15743                                15743 
                Master_mlimitecompra                       Master_mpagado 
                               15743                                15743 
                    Master_fechaalta                   Master_mpagominimo 
                               15743                                15743 
                 Visa_fultimo_cierre                     Visa_delinquency 
                                7285                                 7253 
                         Visa_status            Visa_mfinanciacion_limite 
                                7253                                 7253 
                   Visa_Fvencimiento                     Visa_msaldototal 
                                7253                                 7253 
                    Visa_msaldopesos                   Visa_msaldodolares 
                                7253                                 7253 
                  Visa_mlimitecompra                         Visa_mpagado 
                                7253                                 7253 
                      Visa_fechaalta                     Visa_mpagominimo 
                                7253                                 7253 
          mtarjeta_master_descuentos             mtarjeta_visa_descuentos 
                                4386                                 4381 
                      active_quarter                          cliente_vip 
                                   0                                    0 
                            internet                         cliente_edad 
                                   0                                    0 
                  cliente_antiguedad                        mrentabilidad 
                                   0                                    0 
                mrentabilidad_annual                          mcomisiones 
                                   0       

## Gráficos

### Histograma

In [ ]:
pdf("histogramas.pdf", width = 15, height = 15) 
plot_histogram(dtrain)
dev.off()

### Densidad

In [ ]:
pdf("densidad.pdf", width = 15, height = 15) 
plot_density(dtrain)
dev.off()

### Barras

In [ ]:
pdf("barras.pdf", width = 15, height = 15) 
plot_bar(dtrain, ggtheme = theme_classic())
dev.off()

ERROR: Error in pdf("barras_v2.pdf", width = 15, height = 15, ncol = 1L): unused argument (ncol = 1)


### Barras por clase ternaria

In [25]:
dplot <- dtrain
dplot$clase_ternaria_discreta <- match(dtrain$clase_ternaria, c("CONTINUA", "BAJA+1", "BAJA+2")) - 1
dplot$clase_ternaria_discreta <- factor(dplot$clase_ternaria_discreta)

In [ ]:
pdf("barras-by-clase_ternaria.pdf", width = 15, height = 15) 
plot_bar(dplot, by = "clase_ternaria_discreta")
dev.off()

### Quantile Quantile

<i>Tarda un siglo...</i>

In [ ]:
pdf("quantile-quantile.pdf", width = 15, height = 15) 
plot_qq(dplot)
dev.off()

### Boxplot

<i>Tarda un siglo...</i>

In [ ]:
pdf("boxplot_v2.pdf", width = 15, height = 15) 
#plot_boxplot(dplot, by = "clase_ternaria_discreta", ncol = 2L)
plot_boxplot(dtrain, by = "clase_ternaria", ncol = 1L)
dev.off()

### Scatterplot

<i>Tarda un siglo...</i>

In [ ]:
pdf("scatterplot.pdf", width = 15, height = 15) 
#plot_scatterplot(dplot, by = "clase_ternaria_discreta", sampled_rows = nrow(dplot))
plot_scatterplot(dtrain, by = "clase_ternaria", sampled_rows = nrow(dplot))
dev.off()

### Correlación

<i>Tarda un siglo...</i>

In [ ]:
#pdf("correlograma.pdf", width = 15, height = 15) 
#plot_correlation(dtrain, type = 'continuous' , cor_args = list("method = 'circle'"))
#dev.off()

<i>meter lo que hice yo en RStudio...</i>

In [128]:
#dcor <- cor( select(dtrain, where(~is.numeric(.))) ) 
dcor <- dtrain[, c("numero_de_cliente", "foto_mes", "clase_ternaria") := NULL]

<i>Nos quedamos con aquellos cuya correlación es >= 0.8 </i>

In [134]:
pdf("correlograma.pdf", width = 15, height = 15) 
# Calcula la matriz de correlación
correlation_matrix <- cor(dcor)

# Encuentra las parejas de columnas con correlación mayor o igual a 0.8
high_correlation_pairs <- which(abs(correlation_matrix) >= 0.8 & abs(correlation_matrix) < 1, arr.ind = TRUE)

# Obtén los nombres de las columnas con alta correlación
column_names <- colnames(dcor)

# Extrae los pares de columnas con alta correlación
high_correlation_column_names <- column_names[high_correlation_pairs[, 1]]
high_correlation_column_names <- unique(c(high_correlation_column_names, column_names[high_correlation_pairs[, 2]]))

# Filtra la matriz de correlación para incluir solo las columnas con alta correlación
high_correlation_matrix <- correlation_matrix[high_correlation_column_names, high_correlation_column_names]

# Grafica la matriz de correlación
corrplot(high_correlation_matrix, method = "circle", type = "upper", tl.cex = 0.7, addCoef.col = "black", number.digits = 2)

dev.off()

png 
  2

In [140]:
high_correlation_column_names

[1] "mcomisiones_otras"        "mprestamos_hipotecarios" 
 [3] "cprestamos_hipotecarios"  "mcomisiones"             
 [5] "cforex_sell"              "cforex"                  
 [7] "catm_trx"                 "matm"                    
 [9] "cextraccion_autoservicio" "mextraccion_autoservicio"
[11] "matm_other"               "catm_trx_other"

### PCA

Análisis de componentes principales

Usamos las variables con correlación > 0.8 del paso anterior

In [144]:
df_pca <-  dtrain[, high_correlation_column_names,  with = FALSE]

In [147]:
pca <- prcomp(df_pca, center=TRUE, scale=TRUE)

In [155]:
resumen <- summary(pca) 

In [164]:
resumen

Importance of components:
                          PC1    PC2    PC3    PC4    PC5     PC6     PC7
Standard deviation     1.8961 1.4188 1.4031 1.3523 1.3424 0.65629 0.41678
Proportion of Variance 0.2996 0.1677 0.1641 0.1524 0.1502 0.03589 0.01448
Cumulative Proportion  0.2996 0.4673 0.6314 0.7838 0.9340 0.96985 0.98433
                           PC8     PC9    PC10      PC11     PC12
Standard deviation     0.39730 0.17251 0.02141 1.118e-13 5.42e-14
Proportion of Variance 0.01315 0.00248 0.00004 0.000e+00 0.00e+00
Cumulative Proportion  0.99748 0.99996 1.00000 1.000e+00 1.00e+00

<b><i>Usando la biblioteca DataExplorer</b></i>

In [ ]:
# plot_prcomp(dtrain, maxcat = 5L)
pdf("pca.pdf", width = 15, height = 15) 
plot_prcomp(na.omit(dtrain), variance_cap = 0.9, nrow = 1L, ncol = 1L)
dev.off()

<i>El metodo plot_prcomp tira lo siguiente:</i>
"The following features are dropped due to zero variance:
	* foto_mes
	* cliente_vip
	* ccuenta_corriente
	* mcuenta_corriente_adicional
	* cprestamos_hipotecarios
	* mprestamos_hipotecarios
	* mplazo_fijo_pesos
	* mpayroll2
	* cpayroll2_trx
	* Master_delinquency
	* Master_madelantodolares
	* Master_mpagosdolares
	* Visa_delinquency
	* Visa_madelantodolares
	* Visa_mpagosdolares

# Clientes

Tenemos una serie de datos propios del cliente.
- numero_cliente
- active_quarter
- cliente_vip
- internet
- edad
- antiguedad
- ctrx_quarter

# Ganancias del Banco

Tenemos una serie de datos que se asocian con la ganancia que recibe el banco por parte del cliente.
- ganancia_mes_actual
- ganancia_anual
- comisiones
- ganancia_por_intereses
- ganancia_por_inversiones

# Comisiones

Tenemos una serie de variables asociadas a comisiones que se le cobran al cliente:
- Por mantenimiento 
- Otras

# Acreditaciones

El cliente puede recibir acreditaciones de distintos entes.

# Inversiones

Hay varios tipo de inversiones:
- Inversión tipo I
- Inversión tipo II
- Plazo Fijo

# Cuentas

La cuenta asocia.....

# Productos

Los clientes tienen una serie de productos asociados a una o más cuentas:
- Cajas de Ahorro
- Cuentas Corrientes
- Préstamos
    - Personales
    - Prendarios
    - Hipotecarios
- Seguros
    - Vida
    - Auto
    - Vivienda
    - Caja de seguridad
    - Accidentes personales

## Cajas de ahorro

## Cuentas corrientes

## Préstamos

<p><b>Grafico de barras - prestamos cantidades</b></p>

In [58]:
dtrain_filtrado <- dtrain[clase_ternaria %in% c("BAJA+1", "BAJA+2")]

In [59]:
# Convertir cprestamos_hipotecarios en factor para mostrar todos los valores únicos
dtrain_filtrado$cprestamos_hipotecarios <- as.factor(dtrain_filtrado$cprestamos_hipotecarios)
dtrain_filtrado$cprestamos_personales <- as.factor(dtrain_filtrado$cprestamos_personales)
dtrain_filtrado$cprestamos_prendarios <- as.factor(dtrain_filtrado$cprestamos_prendarios)

In [107]:
pdf("prestamos_cantidades.pdf", width = 15, height = 15) 
# Crear el gráfico de barras
p1 <- ggplot(dtrain_filtrado, aes(x = cprestamos_hipotecarios, fill = clase_ternaria)) +
  geom_bar(position = "dodge", stat = "count") +
  geom_text(stat = 'count', aes(label = ..count..)) +
  labs(x = "cprestamos_hipotecarios", y = "Count") +
  theme_minimal() +
  scale_x_discrete(drop = FALSE)


# Crear el gráfico de barras
p2 <- ggplot(dtrain_filtrado, aes(x = cprestamos_personales, fill = clase_ternaria)) +
  geom_bar(position = "dodge", stat = "count") +
  geom_text(stat = 'count', aes(label = ..count..)) +
  labs(x = "cprestamos_personales", y = "Count") +
  theme_minimal() +
  scale_x_discrete(drop = FALSE)


# Crear el gráfico de barras
p3 <- ggplot(dtrain_filtrado, aes(x = cprestamos_prendarios, fill = clase_ternaria)) +
  geom_bar(position = "dodge", stat = "count") +
  geom_text(stat = 'count', aes(label = ..count..)) +
  labs(x = "cprestamos_prendarios", y = "Count") +
  theme_minimal() +
  scale_x_discrete(drop = FALSE)

grid.arrange(p1, p2, p3, ncol = 1)
dev.off()

png 
  2

<p><b>Histograma - prestamos montos</b></p>

In [91]:
dtrain_filtrado_baja1 <- dtrain_filtrado[clase_ternaria %in% c("BAJA+1")]
dtrain_filtrado_baja2 <- dtrain_filtrado[clase_ternaria %in% c("BAJA+2")]

In [109]:
pdf("prestamos_montos.pdf", width = 15, height = 15) 

p11 <- dtrain_filtrado_baja1%>% as.data.frame %>% ggplot(aes(x = mprestamos_personales)) + ggtitle("mprestamos_personales BAJA+1") + scale_x_continuous(labels = scales::comma) +
  geom_histogram(colour = 10, bins = 30) 
p12 <- dtrain_filtrado_baja2%>% as.data.frame %>% ggplot(aes(x = mprestamos_personales)) + ggtitle("mprestamos_personales BAJA+2") + scale_x_continuous(labels = scales::comma) +
  geom_histogram(colour = "blue", bins = 30) 

p21 <- dtrain_filtrado_baja1%>% as.data.frame %>% ggplot(aes(x = mprestamos_hipotecarios)) + ggtitle("mprestamos_hipotecarios BAJA+1") + scale_x_continuous(labels = scales::comma) +
  geom_histogram(colour = 10, bins = 30) 
p22 <- dtrain_filtrado_baja2%>% as.data.frame %>% ggplot(aes(x = mprestamos_hipotecarios)) + ggtitle("mprestamos_hipotecarios BAJA+2") + scale_x_continuous(labels = scales::comma) +
  geom_histogram(colour = "blue", bins = 30) 

p31 <- dtrain_filtrado_baja1%>% as.data.frame %>% ggplot(aes(x = mprestamos_prendarios)) + ggtitle("mprestamos_prendarios BAJA+1") + scale_x_continuous(labels = scales::comma) +
  geom_histogram(colour = 10, bins = 30) 
p32 <- dtrain_filtrado_baja2%>% as.data.frame %>% ggplot(aes(x = mprestamos_prendarios)) + ggtitle("mprestamos_prendarios BAJA+2") + scale_x_continuous(labels = scales::comma) +
  geom_histogram(colour = "blue", bins = 30) 

grid.arrange(p11, p12, p21, p22, p31, p32, ncol = 2)
dev.off()

png 
  2

## Seguros

# Tarjetas

Las tarjetas están asociadas a ciertos productos y cuentas.
- Tarjetas de débito
- Tarjetas de crédito.
<br>

Tenemos dos <b>tarjeteras</b> principales. Poseen muchos datos, asi que analizarlas por separado.
- Visa
- Master
<br>

A su vez, el cliente puede tener <b>débitos automáticos</b>
- cantidades x mes
- montos x mes
<br>

Pueden haber <b>descuentos</b> por el uso de tarjetas:
- cant. y monto x mes
<br>


El cliente puede tener <b>débitos automáticos</b>:
- cant. y monto x mes

# Transacciones

Hay varias operaciones que el cliente puede hacer. Cada operación tiene sus propios datos asociados.
- Pago de servicios
- Compra y venta de moneda
- Op. por call-center
- Op. por HB
- Transferencias
    - Recibidas
    - Emitidas
- Cheques
    - Depositados
    - Emitidos
- Op. en Cajeros Automáticos
- Op. por la app.
- Op. en caja
    - transacciones
    - consultas
    - depósitos
    - extracciones
    - otras

In [ ]:
## Graficos 

# Feature Engineering

In [ ]:
## Group bottom 20% `clarity` by frequency
group_category(diamonds, feature = "clarity", threshold = 0.2, update = TRUE)

## Group bottom 20% `clarity` by `price`
group_category(diamonds, feature = "clarity", threshold = 0.2, measure = "price", update = TRUE)

## Dummify diamonds dataset
dummify(diamonds)
dummify(diamonds, select = "cut")

## Set values for missing observations
df <- data.frame("a" = rnorm(260), "b" = rep(letters, 10))
df[sample.int(260, 50), ] <- NA
set_missing(df, list(0L, "unknown"))

## Update columns
update_columns(airquality, c("Month", "Day"), as.factor)
update_columns(airquality, 1L, function(x) x^2)

## Drop columns
drop_columns(diamonds, 8:10)
drop_columns(diamonds, "clarity")